<a href="https://colab.research.google.com/github/HNXJ/NeuroPy/blob/main/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# !pip install wget
# !pip install mne
# !pip install dash
import os
import wget
import zipfile


def download(folder_path = ''):

    wget.download("https://github.com/HNXJ/NeuroPy/archive/refs/heads/main.zip")
    with zipfile.ZipFile("NeuroPy-main.zip", 'r') as zip_ref:
        zip_ref.extractall(folder_path)

    try:
        os.mkdir("Data/")
    except:
        pass

    wget.download('link', out=folder_path + "Data/")
    print("All files are downloaded to " + folder_path)
    # return "NeuroPy-main/"


download()
import sys
sys.path.insert(1, "NeuroPy-main/")

ValueError: ignored

In [19]:
import Datasets
import Methods
import Connect
import Viewer

import numpy as np

# NeuroPy

#### Tutorial documentation

### github@HNXJ

Use web browser to open, consider enabling 
JS and trust notebook JS content (for plotly html plots)

### Load dataset

For loading your data, create a Datasets.Dataset() variable class and load your .mat, .csv or other types as a dictionary to Dataset.data. Then put your data parts into Dataset.signals dict() like the example bellow:

In [29]:
import scipy.io as sio

d = Datasets.Dataset()
d.data = sio.loadmat('/content/Data/data.mat')
d.cues = sio.loadmat('Data/cues.mat')['c']

d.signals['pfc'] = d.data['lfp'][:, 0:16, :]
d.signals['p7a'] = d.data['lfp'][:, 16:32, :]
d.signals['v4'] = d.data['lfp'][:, 32:48, :]

ValueError: ignored

For examples in this tutorial, the laminar recording used in [Bastos et al, 2020, layer and rhythm specifity...] has been used. If you had its "data.mat" file, the method Dataset.load_laminar_data(filename) will be able to load id; otherwise use the loading format shown in cell above.

The method "Dataset.print_all_content(dict_)" is a method for checking all content of a dictionary file; use it if you want to check your ".mat", ".npy" and ... file content as a dict. 

In [ ]:
dataset = Datasets.Dataset()
dataset.load_laminar_data(filename="Data/data.mat")
dataset.print_all_content()
trials_block = dataset.get_trials(key='block', l=0, r=600)
trials_trial = dataset.get_trials(key='trial', l=0, r=600)
trials = [i for i in range(0, 600, 1)]

### Event related potential plot

In [ ]:
Methods.ERP_plot(data=dataset.signals['pfc'], title="ERP of pfc for all trials", save=False, filename="ERP_plot")

### Current source density plot

In [ ]:
Methods.CSD_plot(data=dataset.signals['v4'], title="Average CSD for all trials", save=False, filename="ACSD_plot")

### Power spectrum density plots

In [ ]:
trials = [40, 60]
psd, freqs = Methods.power_spectrum_density(data=dataset.signals['pfc'], save=True, t1=1500, t2=4500, fmin=0,
            fmax=100, normalize_w=True, bw=75, k=0, trials=trials)
Methods.customplot(x=psd[:, :, 1].transpose(), save=False, show=True, filename="PSD_plot.html"
                , w=251, h=16, t=freqs, y=[i for i in range(16)], relative=True
                ,xlabel="T", ylabel="A", title="Plot", reverse=True
                ,xtext=False, xtext_labels=["t"])

### PSD ratio plotter

In [ ]:
Methods.psd_ratio_plotter(psd=psd[:, :, 0], freqs=freqs,
                  title="T40SpectBandAvgPower")
Methods.psd_ratio_compare_plotter(psd1=psd[:, :, 1], psd2=psd[:, :, 0],
                          freqs=freqs, title="PSDRatio_T60-T40")

### PSD in time windows

In order to analysis the time and event effects on the behavior and features of the data, methods for time decomposition based analysis are implemented. more methods may be available for connectivity analysis later. As this extracted data may be time consuming, so saving and loading methods are implemented.

In [ ]:
tpsd, freqs, times = Connect.time_power_spectrum_density(data=dataset.signals['pfc']
                                , save=True, bands=True
                                , time_window_size=500, time_overlap=50
                                , trials=trials, bw=50, tl=0, tr=4000, time_base=-1500)

Datasets.save_list([tpsd, freqs, times], "Data/1-600-tpsd.txt")
[tpsd, freqs, times] = Datasets.load_list("Data/1-600-tpsd.txt")

# Average PSD
tpsd_m = np.zeros([tpsd.shape[0], tpsd.shape[1], tpsd.shape[2], 12]) 
for i in range(12):
    tpsd_m[:, :, :, i] = np.mean(tpsd[:, :, :, i*50:(i+1)*50], 3)

### Spectral correlation in time windows

In [ ]:
tsc = Connect.time_spectral_coherence(data=tpsd, trials=trials, ts=times)

### Average TSC
tsc_m = np.zeros([tsc.shape[0], tsc.shape[1], tsc.shape[2], 12]) 
for i in range(12):
    tsc_m[:, :, :, i] = np.mean(tsc[:, :, :, i*50:(i+1)*50], 3)

### Granger causality is time windows

In [ ]:
tgc, times = Connect.time_granger_causality(data=dataset.signals['pfc'], time_window_size=500
                                , time_overlap=0, trials=trials, bw=50, tl=0
                                , tr=3500, time_base=-1500)

Datasets.save_list([tgc, times], "Data/1-600-tgc.txt")
[tgc, times] = Datasets.load_list("Data/1-600-tgc.txt")

### Average TGC
tgc_m = np.zeros([tgc.shape[0], tgc.shape[1], tgc.shape[2], 12]) 
for i in range(12):
    tgc_m[:, :, :, i] = np.mean(tgc[:, :, :, i*5:(i+1)*5], 3)

### Visualization interactive data using dash/plotly web appliction interface

In [ ]:
Viewer.run(data=tpsd, fqs=freqs, title="PSD in time, pfc area ", bands=True
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=trials)

In [ ]:
Viewer.run(data=tsc, fqs=freqs, title="Spectral coherence in time, V4 area ", bands=False
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=trials)

In [ ]:
Viewer.run(data=tgc, fqs=freqs, title="Granger causality in time, pfc area "
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=trials)

In [ ]:
Viewer.run(data=tpsd_m, fqs=freqs, title="PSD in time, pfc area ", bands=True
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=[i for i in range(12)])

In [ ]:
Viewer.run(data=tsc_m, fqs=freqs, title="Spectral coherence in time, V4 area ", bands=False
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=[i for i in range(12)])

In [ ]:
Viewer.run(data=tgc_m, fqs=freqs, title="Granger causality in time, pfc area "
            , xlabel="Frequency", ylabel="Channel", fr=times, tr=[i for i in range(12)])

* Other examples coming soon ...

# End